# Check correctness on test data set
## Check tools
First we check we have the required tools.

In [ ]:
%run Tools.ipynb

## Check formulae
We check that the data_check and formulae directories exist.

In [ ]:
mkdir -p data_check formulae

Now generate formulae if they are missing.

In [ ]:
import os
if not os.path.exists('formulae/random_nd.ltl'):
    %run Formulae.ipynb

## Get tool chains
Now we get the tool chains for PBSand Spot complementation via semi-determinization.

In [ ]:
from tools import get_tools
from ltlcross_runner import LtlcrossRunner

In [ ]:
### Tools ###
tools = get_tools(full=False)

print(tools)

### Numbers to measure ###
cols = ['states','edges','transitions','nondet_states','exit_status','time','acc']

## Run ltlcross on test formulae
Finally, we prepare and run the ltlcross runners for our test formulae.

In [ ]:
runners = {}
for source in ('literature','random'):
    for t in ('det','cd','sd','nd'):
        name = '{}_{}'.format(source,t)
        runners[name] = \
            LtlcrossRunner(tools,\
                    res_filename='data_check/{}.csv'.format(name),\
                    formula_files=['formulae/{}.ltl'.format(name)],\
                    cols=cols)

In [ ]:
rerun = True

In [ ]:
from datetime import datetime

for name in runners.keys():
    print('{}: Working on {}'.format(datetime.now().strftime('[%d.%m.%Y %T]'),name))
    if rerun:
        runners[name].run_ltlcross(timeout='120',
                check=True)
    runners[name].parse_results()

In [ ]:
for name, r in runners.items():
    print(f'{name} total timeouts: {r.get_error_count(err_type="timeout")}')
    print(f'{name} total incorrect: {r.get_error_count(err_type="incorrect")}')

In [ ]:
from ltlcross_runner import hunt_error_types

for name in runners.keys():
    print(f'{datetime.now().strftime("[%d.%m.%Y %T]")}: Looking for errors in {name}')
    errors, formulae, tools = hunt_error_types(f'data_check/{name}.log')
    
    if not errors:
        print('No errors!')
    else:
        print(f'Errors: {errors}')
        print(f'Problematic formulae: {formulae}')
        print(f'Tools used: {tools}')

## Run ltlcross on test automata
Now we turn our attention to random automata generated for our evaluation.

## Automata check
Again, we first check that the automata are already generated.

In [ ]:
mkdir -p data_check automata

In [ ]:
import os
if not os.path.exists('automata/random_ba.ltl'):
    %run Automata.ipynb

## Get automata tool chain
Fetch the tool chain for testing on automata.

In [ ]:
### Tools ###
tools = get_tools(full=False, automata=True)

print(tools)

### Numbers to measure ###
cols = ['states','edges','transitions','nondet_states','exit_status','time','acc']

## Run ltlcross on test automata
Finally, we prepare and run the ltlcross runners for our test automata.

In [ ]:
runners = {}
for aut_type in ['random_ba']:
    name = '{}'.format(aut_type)
    runners[name] = \
        LtlcrossRunner(tools,\
                res_filename='data_check/{}.csv'.format(name),\
                formula_files=['automata/{}.ltl'.format(name)],\
                cols=cols)

In [ ]:
from datetime import datetime

for name in runners.keys():
    print('{}: Working on {}'.format(datetime.now().strftime('[%d.%m.%Y %T]'),name))
    if True:
        runners[name].run_ltlcross(timeout='120',
                check=True)
    runners[name].parse_results()

In [ ]:
for name, r in runners.items():
    print(f'{name} total timeouts: {r.get_error_count(err_type="timeout")}')
    print(f'{name} total incorrect: {r.get_error_count(err_type="incorrect")}')

In [ ]:
from ltlcross_runner import hunt_error_types

for name in runners.keys():
    print(f'{datetime.now().strftime("[%d.%m.%Y %T]")}: Looking for errors in {name}')
    errors, formulae, tools = hunt_error_types(f'data_check/{name}.log')
    
    if not errors:
        print('No errors!')
    else:
        print(f'Errors: {errors}')
        print(f'Problematic formulae: {formulae}')
        print(f'Tools used: {tools}')